In [8]:
import os
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data, datasets
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the LSTM-based model class
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout if n_layers > 1 else 0.0)
        # Linear layer input dimension depends on bidirectionality
        fc_input_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.fc = nn.Linear(fc_input_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.bidirectional = bidirectional

    def forward(self, text, text_lengths):
        # text: [sentence_length, batch_size]; text_lengths: [batch_size]
        embedded = self.dropout(self.embedding(text))
        # Pack the sequence of embeddings
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        # Unpack (pad) the sequence (output not used further here)
        nn.utils.rnn.pad_packed_sequence(packed_output)
        # hidden shape: [n_layers * num_directions, batch_size, hidden_dim]
        if self.bidirectional:
            # Concatenate the final forward and backward hidden states
            hidden_combined = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            # Use the final hidden state (from the last layer)
            hidden_combined = hidden[-1,:,:]
        # Apply dropout to final hidden state and pass through the linear layer
        hidden_dropped = self.dropout(hidden_combined)
        return self.fc(hidden_dropped)

# Accuracy calculation
def multiclass_accuracy(preds, y):
    """
    preds: raw logits of shape [batch, n_classes]
    y:     ground-truth indices, shape [batch]
    """
    predicted_classes = preds.argmax(dim=1)
    correct = (predicted_classes == y).float()
    return correct.sum() / len(correct)

# Training and evaluation functions
def train_epoch(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.label)
        acc  = multiclass_accuracy(predictions, batch.label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_epoch(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths)
            loss = criterion(predictions, batch.label)
            acc = multiclass_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed = end_time - start_time
    mins = int(elapsed // 60)
    secs = int(elapsed % 60)
    return mins, secs

In [ ]:
import torch
import torch.nn as nn

# ------------------------------------------------------------
# 1. Choose a device once
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# ------------------------------------------------------------
# 2. Reload the torch-text Fields
# ------------------------------------------------------------
TEXT  = torch.load("TEXT_field.pth",  weights_only=False)
LABEL = torch.load("LABEL_field.pth", weights_only=False)

# ------------------------------------------------------------
# 3. Re-create the model
# ------------------------------------------------------------
INPUT_DIM     = len(TEXT.vocab)
EMBEDDING_DIM = 4500
HIDDEN_DIM    = 256
OUTPUT_DIM    = len(LABEL.vocab)
N_LAYERS      = 3
BIDIRECTIONAL = True
DROPOUT       = 0.7
PAD_IDX       = TEXT.vocab.stoi[TEXT.pad_token]

print(PAD_IDX)
print("============")

model = LSTMClassifier(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM,
                       OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL,
                       DROPOUT, pad_idx=PAD_IDX)

# load weights, then move the whole model to the chosen device
model.load_state_dict(torch.load("best_acc.pt", map_location=device))
model = model.to(device).eval()



Using: cuda


In [10]:

# 3. Build your example and move it
example_sentence = "Great i love it "

tokens   = [tok.lower() for tok in TEXT.tokenize(example_sentence)]
indices  = [TEXT.vocab.stoi[t] if t in TEXT.vocab.stoi
            else TEXT.vocab.stoi[TEXT.unk_token] for t in tokens]

text_tensor = torch.LongTensor(indices).unsqueeze(1).to(device)  # [seq_len, batch=1]
text_length = torch.LongTensor([len(indices)]).to(device)        # [batch=1]

# 4. Forward pass
with torch.no_grad():
    output = model(text_tensor, text_length)
    prediction = output.argmax(dim=1).item()

predicted_label = LABEL.vocab.itos[prediction]

print(f"Predicted label: {predicted_label}")

Predicted label: 1
